In [1]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)
from multilayer_model.m2_multi_bce import BCEcVAE
from multilayer_model.m2_mutli_mse import MSEcVAE
from multilayer_loss.l_multi2_bce import l2_bce
from multilayer_loss.l_multi2_mse import l2_mse
from vae_earlystopping import EarlyStopping

SyntaxError: invalid syntax (m2_multi_bce.py, line 23)

In [ ]:
x_data = np.load('./data/metal.npy')      # Target (X)
x2_data = np.load('./data/support_norm.npy')
c_data = np.load('./data/pre_re_change_temp_logconst.npy') # Condition (C)

# 데이터 분할 (Train/Val/Test)
c_train, c_temp, x_train, x_temp, x2_train, x2_temp = train_test_split(c_data, x_data,x2_data, test_size=0.4, random_state=42)
c_val, c_test, x_val, x_test, x2_val, x2_test = train_test_split(c_temp, x_temp,x2_temp, test_size=0.5, random_state=42)

# 스케일링
c_scaler, x_scaler,x2_scaler = MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
c_train = c_scaler.fit_transform(c_train)
x_train = x_scaler.fit_transform(x_train)
x2_train = x2_scaler.fit_transform(x2_train)
c_val, x_val, x2_val = c_scaler.transform(c_val), x_scaler.transform(x_val), x2_scaler.transform(x2_val)
c_test, x_test, x2_test = c_scaler.transform(c_test), x_scaler.transform(x_test), x2_scaler.transform(x2_test)

# 텐서 변환
def to_tensor(arr): return torch.tensor(arr, dtype=torch.float32)
c_train, x_train, x2_train = to_tensor(c_train), to_tensor(x_train), to_tensor(x2_train)
c_val, x_val, x2_val = to_tensor(c_val), to_tensor(x_val), to_tensor(x2_val)
c_test, x_test, x2_test = to_tensor(c_test), to_tensor(x_test), to_tensor(x2_test)

train_loader = DataLoader(TensorDataset(x_train, x2_train, c_train), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val, x2_val, c_val), batch_size=64, shuffle=False)
test_loader = DataLoader(TensorDataset(x_test, x2_test, c_test), batch_size=64, shuffle=False)

x,x2, c = next(iter(train_loader))
x_dim = x.shape[1]
x2_dim = x2.shape[1]
c_dim = c.shape[1]
z_dim = 8
z2_dim = 8
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 3. BCE 및 MSE 모델 정의
model_bce = BCEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)
model_mse = MSEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)

In [ ]:
optimizer_bce = optim.Adam(model_bce.parameters(), lr=1e-3, weight_decay=1e-5)
optimizer_mse = optim.Adam(model_mse.parameters(),lr=1e-3, weight_decay = 1e-5)
es_bce= EarlyStopping(patience=40, min_delta=1e-9)
es_mse = EarlyStopping(patience=40,min_delta=1e-9)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_runs = 20
r2_results = []

try:
    x_data = np.load('./data/metal.npy')
    x2_data = np.load('./data/support_norm.npy')
    c_data = np.load('./data/pre_re_change_temp_logconst.npy')
    print(f"✅ Data loaded successfully. Device: {device}")
except FileNotFoundError as e:
    print(f"❌ Error: {e}. 데이터 파일 경로를 확인해주세요.")
    # 실행 중단을 방지하기 위한 예시 데이터 (실제 실행 시 삭제)
    # x_data, x2_data, c_data = np.random.rand(100, 10), np.random.rand(100, 5), np.random.rand(100, 3)

# ----------------------------------------------------------------
# 2. 학습 함수 정의 (중복 코드 방지)
# ----------------------------------------------------------------
def train_model(model, optimizer, criterion, train_loader, val_loader, es, epochs=800):
    for epoch in range(1, epochs + 1):
        model.train()
        for x, x2, c in train_loader:
            x, x2, c = x.to(device), x2.to(device), c.to(device)
            optimizer.zero_grad()
            out, mu,mu2, logvar,logvar2 = model(x, x2, c)
            loss = criterion(out, x2, mu,mu2, logvar, logvar2)['loss']
            loss.backward()
            optimizer.step()
        
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for vx, vx2, vc in val_loader:
                vx, vx2, vc = vx.to(device), vx2.to(device), vc.to(device)
                vh, vm,vm2, vl,vl2 = model(vx, vx2, vc)
                v_loss += criterion(vh, vx2, vm, vm2, vl,vl2)['loss'].item()
        
        avg_v_loss = v_loss / len(val_loader)
        if es(avg_v_loss, model):
            break
    es.load_best_model(model)

# ----------------------------------------------------------------
# 3. 메인 루프 (20회 반복)
# ----------------------------------------------------------------
print(f"Starting {num_runs} independent runs...")

for run in range(1, num_runs + 1):
    # 데이터 분할
    c_train, c_temp, x_train, x_temp, x2_train, x2_temp = train_test_split(
        c_data, x_data, x2_data, test_size=0.4, random_state=run
    )
    c_val, c_test, x_val, x_test, x2_val, x2_test = train_test_split(
        c_temp, x_temp, x2_temp, test_size=0.5, random_state=run
    )

    # 스케일링
    c_scaler, x_scaler, x2_scaler = MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
    c_train = c_scaler.fit_transform(c_train)
    x_train = x_scaler.fit_transform(x_train)
    x2_train = x2_scaler.fit_transform(x2_train)
    
    c_val, x_val, x2_val = c_scaler.transform(c_val), x_scaler.transform(x_val), x2_scaler.transform(x2_val)
    c_test, x_test, x2_test = c_scaler.transform(c_test), x_scaler.transform(x_test), x2_scaler.transform(x2_test)

    # 텐서 및 데이터로더
    def to_t(arr): return torch.tensor(arr, dtype=torch.float32)
    train_loader = DataLoader(TensorDataset(to_t(x_train), to_t(x2_train), to_t(c_train)), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(to_t(x_val), to_t(x2_val), to_t(c_val)), batch_size=64, shuffle=False)
    test_loader = DataLoader(TensorDataset(to_t(x_test), to_t(x2_test), to_t(c_test)), batch_size=64, shuffle=False)

    # 모델 초기화 (클래스 정의가 사전에 되어 있어야 함)
    x_dim, x2_dim, c_dim = x_train.shape[1], x2_train.shape[1], c_train.shape[1]
    z_dim, z2_dim = 8, 8

    model_bce = BCEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)
    model_mse = MSEcVAE(x_dim, x2_dim, c_dim, z_dim, z2_dim).to(device)
    
    opt_bce = optim.Adam(model_bce.parameters(), lr=1e-3, weight_decay=1e-5)
    opt_mse = optim.Adam(model_mse.parameters(), lr=1e-3, weight_decay=1e-5)
    
    # 학습 진행
    train_model(model_bce, opt_bce, l2_bce, train_loader, val_loader, EarlyStopping(patience=40))
    train_model(model_mse, opt_mse, l2_mse, train_loader, val_loader, EarlyStopping(patience=40))

    # 테스트 평가
    model_bce.eval()
    model_mse.eval()
    all_pred, all_true = [], []

    with torch.no_grad():
        for xt, xt2, ct in test_loader:
            xt, xt2, ct = xt.to(device), xt2.to(device), ct.to(device)
            b_logit, _, _ ,_,_= model_bce(xt, xt2, ct)
            b_prob = torch.sigmoid(b_logit)
            m_val, _, _ ,_,_= model_mse(xt, xt2, ct)
            
            combined_s = b_prob * m_val
            all_pred.append(combined_s.cpu().numpy())
            all_true.append(xt2.cpu().numpy())

    y_true = np.concatenate(all_true).ravel()
    y_pred = np.concatenate(all_pred).ravel()
    r2 = r2_score(y_true, y_pred)
    r2_results.append(r2)
    print(f"▶ Run {run}/{num_runs} - R2 Score: {r2:.6f}")

# ----------------------------------------------------------------
# 4. 결과 저장 및 출력
# ----------------------------------------------------------------
results_json = {
    "mean_r2": float(np.mean(r2_results)),
    "std_r2": float(np.std(r2_results)),
    "all_r2": [float(x) for x in r2_results]
}

with open("r2_statistics_new.json", "w") as f:
    json.dump(results_json, f, indent=4)

print("\n" + "="*30)
print("📊 FINAL STATISTICS")
print(f"Mean R2: {results_json['mean_r2']:.6f} ± {results_json['std_r2']:.6f}")
print("="*30)

EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 3 out of 40
EarlyStopping counter: 1 out of 40
EarlyStopping counter: 2 out of 40
EarlyStopping counter: 1 out of 40
